<a href="https://colab.research.google.com/github/RajuDasa/agents/blob/foundations_branch/1_foundations/community_contributions/raju/Foundations_Lab5_exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Exercise: Agent Loop - Recommand or Buy/sell a stock

In [ ]:
!pip install litellm

In [ ]:
from google.colab import userdata
from litellm import completion
import os

os.environ["GEMINI_API_KEY"] = userdata.get('GEMINI_API_KEY')


In [ ]:
!pip install yfinance

In [ ]:
import yfinance as yf
import datetime
from typing import List, Dict, Any

JSON = Dict[str, Any]

def get_stock_price(name:str, start_date:datetime, end_date:datetime) -> List[JSON]:
  dat = yf.Ticker(name) #"RELIANCE.NS"
  history = dat.history(start=start_date, end=end_date) #"2025-12-01", "2025-12-31"
  return history['Close'].reset_index().to_json(orient="records", date_format="iso")

In [ ]:
#get_stock_price("TCS.NS","2025-12-01","2025-12-31") # NS = NIFTY Stock exchange


In [ ]:
def stock_exchange(stock_name:str, transact:str) -> str:
  if transact == "buy":
    print("Brought stock")
  elif transact == "sell":
    print("Sold stock")
  else:
    print("Transaction failed")
    return "Failed. expected 'buy' or 'sell' for transact argument"

  return "success"

In [ ]:
import datetime

current_date = datetime.datetime.today().strftime("%Y-%m-%d")

system_prompt = f"""
You are a stock analyst agent, who either recommand or buy/sell the given stock based on closing price of last 2 weeks.
Only provide suggestion if user asks for it explicitly, otherwise buy/sell the stock by default.
Current date: {current_date}
Average the close prices of last week and second last week seperately,
Find price difference between these averages and buy if it is positive number else sell.
#use the tools:

**get_stock_price**
purpose: fetch the closing prices for a week duration for a given stock.

**stock_exchange**
purpose: buy or sell a given stock

Prepare a plan and execute step by step.
Think before act.
All dates will be in YYY-MM-DD format.
You handle only stocks and nothing else, inform the same for unrelated task request.
Don't ask any questions, go with your intuition.

Finally explain the steps you took and final action you took (buy or sell)
"""

In [ ]:
#from function_schema import get_function_schema

func_list = {
    "get_stock_price" : get_stock_price,
    "stock_exchange" : stock_exchange
}

tools = []

# for func in func_list.values():
#   schema = get_function_schema(func) #err: isinstance() arg 2 must be a type, a tuple of types, or a union
#   tools.append({"type": "function", "function": schema})

get_stock_price_schema = {
    "name": "get_stock_price",
    "description": "fetch the closing prices for a week duration for a given stock.",
    "parameters": {
        "type": "object",
        "properties": {
            "name": {
                "type": "string",
                "description": "The ticker symbol of the stock."
            },
            "start_date": {
                "type": "string",
                "description": "The start date in YYYY-MM-DD format."
            },
            "end_date": {
                "type": "string",
                "description": "The end date in YYYY-MM-DD format."
            }
        },
        "required": ["name", "start_date", "end_date"]
    }
}
tools.append({"type": "function", "function": get_stock_price_schema})

# Use get_function_schema for stock_exchange as its type hints are simpler
stock_exchange_schema = {
    'name': 'stock_exchange',
    'description': "buy or sell a stock",
    'parameters': {
        'type': 'object',
        'properties': {
            'stock_name': {
                'type': 'string',
                'description': 'The ticker symbol of the stock.'
                },
            'transact': {
                'type': 'string',
                'description': 'indicate buy or sell using values: "buy", "sell"'
                }
        },
        'required': ['stock_name', 'transact']
    }
}
tools.append({"type": "function", "function": stock_exchange_schema})


In [ ]:
import json

def call_tools(tools):
  msgs=[]
  for tool in tools:
    func_name = tool.function.name
    args = json.loads(tool.function.arguments)
    func = func_list.get(func_name)
    result = func(**args) if func else {}
    msgs.append({'role':'tool','content':json.dumps(result), 'tool_call_id':tool.id})

  return msgs


In [ ]:
def agent_loop(msgs):
  stop = False
  choice = []
  while not stop:
    response = completion("gemini/gemini-2.5-flash-lite", messages=msgs, tools=tools)
    choice = response.choices[0]
    if choice.finish_reason == 'tool_calls':
      outputs = call_tools(choice.message.tool_calls)
      msgs.append(choice.message)
      msgs.extend(outputs)
    else:
      stop = True

    print(choice.message.content)


In [ ]:
# import litellm

# os.environ["LITELLM_LOG"] = "ERROR"
# #litellm._turn_on_debug()  #don't turn on

# #To turn off debug: --not working
# litellm.set_verbose = False
# litellm.suppress_debug_info = True


In [ ]:
#Testcase 1
user_prompt = "get temperature for new york city"
messages = [{"role":"system", "content": system_prompt},{"role":"user", "content":user_prompt}]
agent_loop(messages)

#output:
#I am a stock analyst agent, I can only handle stocks and nothing else. I cannot get the temperature for New York City.

In [ ]:
#Testcase 2
user_prompt = 'Handle "RELIANCE.NS" stock for me'
messages = [{"role":"system", "content": system_prompt},{"role":"user", "content":user_prompt}]
agent_loop(messages)

#output:
# step1: I will first get the closing prices for Reliance Industries Limited for the past two weeks. Then, I will calculate the average closing price for each week. After that, I will find the difference between the two weekly averages. Finally, I will buy the stock if the difference is positive and sell it if the difference is negative.\n"
# step2: I have retrieved the closing prices for RELIANCE.NS for the past two weeks.\n\nFor the week of January 8th to January 14th, the closing prices were 1475.30, 1483.20, 1452.80, 1458.80, and 1458.80. The average closing price for this week is (1475.30 + 1483.20 + 1452.80 + 1458.80 + 1458.80) / 5 = 1465.78.\n\nFor the week of January 15th to January 21st, the closing prices were 1457.90, 1413.60, 1394.00, 1404.60, and 1402.50. The average closing price for this week is (1457.90 + 1413.60 + 1394.00 + 1404.60 + 1402.50) / 5 = 1414.52.\n\nThe difference between the two weekly averages is 1414.52 - 1465.78 = -51.26.\n\nSince the difference is negative, I will sell the stock.\n"
# step3: Sold stock
# Final: I have analyzed the closing prices of RELIANCE.NS for the past two weeks. The average closing price for the first week was 1465.78, and for the second week, it was 1414.52. The difference of -51.26 indicates a downward trend, so I have sold the stock.


In [ ]:
#Testcase 3
user_prompt = 'suggest me if I should buy or sell "TCS.NS" stock. Don\'t initiate buy or sell at your end, only provide suggestion'
messages = [{"role":"system", "content": system_prompt},{"role":"user", "content":user_prompt}]
agent_loop(messages)

#output:
# Here's the plan to determine whether you should buy or sell TCS.NS stock:

# 1. **Fetch Stock Prices:** Get the closing prices for the last two weeks for TCS.NS.
#     * Week 1: From 2026-01-09 to 2026-01-16
#     * Week 2: From 2026-01-16 to 2026-01-23
# 2. **Calculate Average Closing Prices:** Compute the average closing price for each of the two weeks.
# 3. **Determine Trend:** Calculate the difference between the average closing price of the second week and the first week.
# 4. **Provide Suggestion:** Based on the difference, suggest whether to buy or sell. If the difference is positive, suggest buying. If it's negative, suggest selling.

# Let's start by executing step 1.

# I have fetched the closing prices for TCS.NS for the past two weeks. Now, I will proceed to calculate the average closing prices for each week and then determine the suggestion.

# I have already fetched the stock prices in the previous step. I will now proceed with calculating the average closing prices and providing the suggestion.
# The average closing price for the first week (2026-01-09 to 2026-01-16) is 3152.50.
# The average closing price for the second week (2026-01-16 to 2026-01-23) is 3147.08.
# The difference between the average closing prices of the second week and the first week is -5.42.
# Since the difference is negative, I suggest you **sell** the TCS.NS stock.
